# Tweets from bioRxiv and medRxiv

Find and display events from the CSHL DOI prefix 10.1101 from the last 7 days, and find the most tweeted preprints.

In [1]:
import sys
sys.path.append('../')
sys.path.insert(0, '..')

import pandas as pd # data analysis library
import json
import mrced2 # module to run event data queries
import os # some file manipulations
import math # some number manipulations
import altair.vegalite.v3 as alt # some data visualizations
from IPython.display import Markdown as md # some markdown manipulations
from datetime import datetime, date, timedelta # some date manipulations

In [2]:
email = "info@front-matter.io"
prefix = "10.1101"
start_date = (date.today() - timedelta(days = 7)).strftime('%Y-%m-%d')
end_date = date.today()

In [3]:
ed = mrced2.eventData(email = email)
ed.buildQuery({'obj-id.prefix' : prefix, 'source': 'twitter', 'rows': 0,'from-occurred-date' : start_date, 'until-occurred-date' : end_date})

https://api.eventdata.crossref.org/v1/events?mailto=Anonymous&obj-id.prefix=10.1101&source=twitter&rows=0&from-occurred-date=2021-07-19&until-occurred-date=2021-07-26


In [4]:
ed.runQuery(retry = 5)

Event Data query started...
API query complete  200
output file written to 1101/tweets.json


In [5]:
pages = math.ceil(ed.events.getHits() / 1000)

13139 events found


In [6]:
email = "info@front-matter.io"
prefix = "10.1101"
start_date = date.today() - timedelta(days = 7)
end_date = date.today()

# find the all result pages for the search
ed = mrced2.eventData(email = email)
ed.getAllPages(pages, {'rows': 1000, 'obj-id.prefix' : prefix, 'source': 'twitter', 'from-occurred-date' : start_date, 'until-occurred-date' : end_date}, fileprefix = '1101/tweets_') 

https://api.eventdata.crossref.org/v1/events?mailto=Anonymous&rows=1000&obj-id.prefix=10.1101&source=twitter&from-occurred-date=2021-07-19&until-occurred-date=2021-07-26
Event Data query started...
API query complete  200
output file written to 1101/tweets_0000.json
https://api.eventdata.crossref.org/v1/events?mailto=Anonymous&cursor=35281021-ffaf-460f-89bd-4db8232e7ef7&rows=1000&obj-id.prefix=10.1101&source=twitter&from-occurred-date=2021-07-19&until-occurred-date=2021-07-26
Event Data query started...
API query complete  200
output file written to 1101/tweets_0001.json
https://api.eventdata.crossref.org/v1/events?mailto=Anonymous&cursor=02017bdf-575c-45e6-9233-8690717a4b2a&rows=1000&obj-id.prefix=10.1101&source=twitter&from-occurred-date=2021-07-19&until-occurred-date=2021-07-26
Event Data query started...
API query complete  200
output file written to 1101/tweets_0002.json
https://api.eventdata.crossref.org/v1/events?mailto=Anonymous&cursor=78b76350-95f3-4462-873c-0c36105df1fc&rows=

Initialisation to look at the properties of the results.

In [7]:
jd1 = mrced2.eventRecord() # instance of a class to interpret the events
files = os.listdir('1101') # get all the filenames

jd1.mergeJsons(files, folder = '1101') # load the json event data from multiple files

failed to load .DS_Store
failed to load .gitkeep
failed to load .ipynb_checkpoints
failed to load preprint_tweets_2021-07-26.csv
output file written to 1101/tweets.json


In [8]:
js = json.load(open("1101/tweets.json"))
df = pd.json_normalize(js, record_path = ['message', 'events'])
gdf = df.groupby(['obj_id']).size().reset_index(name='count').sort_values('count', ascending=False)
cdf = gdf[gdf['count'] >= 3]

In [9]:
email = "info@front-matter.io"
rest = mrced2.restApi(email = email)

data = []
for index, row in cdf.iterrows():
    rest.runQuery(row)
    if rest.work is not None and date.fromisoformat(rest.work["posted"]) >= start_date:
        data.append(rest.work)
print(data)
    
tdf = pd.DataFrame(data, columns=['doi','tweets','archive','subject-area','covid','title','authors','abstract','posted'])
tdf.to_csv('1101/preprint_tweets_' + str(date.today().strftime('%Y-%m-%d')) + '.csv')

tdf.head(50)

REST API query started for 10.1101/2021.07.14.21260307...
REST API query complete  200
REST API query started for 10.1101/2021.02.23.432474...
REST API query complete  200
REST API query started for 10.1101/2021.06.28.21259420...
REST API query complete  200
REST API query started for 10.1101/2021.07.19.21260767...
REST API query complete  200
REST API query started for 10.1101/2021.06.17.448820...
REST API query complete  200
REST API query started for 10.1101/2021.06.01.21258176...
REST API query complete  200
REST API query started for 10.1101/2021.07.15.21260561...
REST API query complete  200
REST API query started for 10.1101/2021.05.16.21257255...
REST API query complete  200
REST API query started for 10.1101/2021.05.31.21258122...
REST API query complete  404
REST API query started for 10.1101/2021.07.05.21260050...
REST API query complete  200
REST API query started for 10.1101/2021.05.08.443253...
REST API query complete  200
REST API query started for 10.1101/2021.07.21.451

,doi,tweets,archive,subject-area,covid,title,authors,abstract,posted
0,10.1101/2021.07.19.21260767,542,medRxiv,Pediatrics,True,Children with SARS-CoV-2 in the National COVID...,"[{'name': 'Blake Martin'}, {'name': 'Peter E. ...",<sec><title>Importance</title><p>SARS-CoV-2</p...,2021-07-22
1,10.1101/2021.07.15.21260561,405,medRxiv,Infectious Diseases (except HIV/AIDS),True,Viral Load of SARS-CoV-2 in Respiratory Aeroso...,"[{'name': 'Kristen K. Coleman'}, {'name': 'Dou...",<sec><title>Background</title><p>Multiple SARS...,2021-07-19
2,10.1101/2021.07.21.451321,219,bioRxiv,Microbiology,True,"Unlike Chloroquine, mefloquine inhibits SARS-C...","[{'name': 'Carolina Q. Sacramento'}, {'name': ...",<p>Repositioning of clinical approved drugs co...,2021-07-21
3,10.1101/2021.07.19.452771,50,bioRxiv,Immunology,True,Comparison of Neutralizing Antibody Titers Eli...,"[{'name': 'Takuya Tada'}, {'name': 'Hao Zhou'}...",<p>The increasing prevalence of SARS-CoV-2 var...,2021-07-19
4,10.1101/2021.07.23.453070,49,bioRxiv,Evolutionary Biology,False,Regulation of sedimentation rate shapes the ev...,"[{'name': 'Omaya Dudin'}, {'name': 'Sébastien ...",<p>Significant increases in sedimentation rate...,2021-07-23
5,10.1101/2021.07.23.453521,35,bioRxiv,Immunology,True,Probiotic consortia improve anti-viral immunit...,"[{'name': 'J Lehtinen Markus'}, {'name': 'Kuma...",<p>Probiotics have been suggested as one solut...,2021-07-23
6,10.1101/2021.07.23.453573,34,bioRxiv,Neuroscience,False,Strong and opponent contributions of dorsomedi...,"[{'name': 'Scott Bolkan'}, {'name': 'Iris Ston...",<p>A classic view of the striatum holds that a...,2021-07-25
7,10.1101/2021.07.19.453013,27,bioRxiv,Evolutionary Biology,False,Phylogenomic analyses of echinoid diversificat...,"[{'name': 'Nicolas Mongiardino Koch'}, {'name'...",<p>Echinoids are key components of modern mari...,2021-07-20
8,10.1101/2021.07.19.452969,20,bioRxiv,Genetics,False,A Bayesian model selection approach to mediati...,"[{'name': 'Wesley L Crouse'}, {'name': 'Gregor...",<p>Mediation analysis is a powerful tool for d...,2021-07-20
9,10.1101/2021.07.22.21260852,19,medRxiv,Transplantation,True,Comparison of infected and vaccinated transpla...,"[{'name': 'Xavier Charmetant'}, {'name': 'Maxi...","<p>Transplant recipients, which receive therap...",2021-07-24


### Tweets of bioRxiv and medRxiv preprints

In [10]:
num_rows = tdf['archive'].count()
num_covid = tdf['covid'].value_counts(ascending=True)[1]
num_biorxiv = tdf['archive'].value_counts(ascending=True)[1]
num_medrxiv = tdf['archive'].value_counts(ascending=True)[0]
end_date = date.today().strftime('%Y-%m-%d')
max_count = tdf['tweets'].max()

md('{} preprints (including {} covering SARS-CoV-2, {} from bioRxiv and {} from medRxiv) published in the last 7 days before {} had been tweeted at least 3 times (maximum {}).'.format(num_rows, num_covid, num_biorxiv, num_medrxiv, end_date, max_count))

42 preprints (including 10 covering SARS-CoV-2, 31 from bioRxiv and 11 from medRxiv) published in the last 7 days before 2021-07-26 had been tweeted at least 3 times (maximum 542).